***Welcome to Medical image processing in Python***<br/>

Presented by: Reza Saadatyar (2024-2025) <br/>
E-mail: Reza.Saadatyar@outlook.com 

**Import the require library**

In [1]:
import os
import sys
import glob
import random
import numpy as np
from tqdm import tqdm
from PIL import Image
from Functions import Data_path
import matplotlib.pyplot as plt
from keras import preprocessing
from skimage import transform
from skimage import io
# from skimage.viewer import ImageViewer
# from OOP import Pre_Processing_R2g
from Functions import image_resizer

**1. Set Image Path & Load data**

In [2]:
folder_path = "D:/Medical-Image-Processing/Data/Inputs/"
files_inputs, _, _ = Data_path.data_path(folder_path, data_format="tif")

folder_path = "D:/Medical-Image-Processing/Data/Masks/"
files_masks, _, _ = Data_path.data_path(folder_path, data_format="TIF")

*Image-Width: n*<br/>
*Image-Height: m*<br/>
*Channels: c*<br/>
*Planes: p*<br/>
*Grayscale: (p, m, n)*<br/>
*RGB: (p, m, n, c)*<br/>

In [3]:
data = io.imread(files_inputs[0])   # Load a file for obtaing size data
img_height = data.shape[0]
img_width = data.shape[1]
img_channels = data.shape[2]
inputs = np.zeros((len(files_inputs), img_height, img_width, img_channels), dtype = np.uint8) # 58*768*896*3
labels = np.zeros((len(files_inputs), img_height, img_width, 2), dtype = bool)                    # 58*768*896*2

In [4]:
sys.stdout.flush()
for ind, _ in tqdm(enumerate(files_inputs)):
    inputs[ind] = io.imread(files_inputs[ind])[:, :, :3]
    mask = np.squeeze(io.imread(files_masks[ind])).astype(bool)
    labels[ind, :, :, 0] = ~mask
    labels[ind, :, :, 1] = mask

58it [00:01, 34.58it/s]


**[Image Resizing](https://scikit-image.org/docs/stable/api/skimage.transform.html#skimage.transform.resize)**<br/>
`Standardizing Image Dimensions:`
Reason: Machine learning models, like CNNs, need input data with fixed dimensions. For instance, if a model requires images of size 224x224x3, all input images must be resized to that shape.<br/>
`Reducing Computational Load:`Resizing images to smaller dimensions lowers computational costs, particularly with large datasets, and aids in faster training or inference for deep learning models.

In [9]:
# Create the resizer instance
resizer = image_resizer.ImageResizer(img_height_resized=255, img_width_resized=255, img_channels=3)

# Resizing the images
resized_images = resizer.resize_images(inputs)
print(f"Resizing images from {inputs.shape} to {resized_images.shape}") 

Resizing images from (58, 768, 896, 3) to (58, 255, 255, 3)


**Augmentation, Re_Color, & Im_Saving**<br/>